## Loading packages

In [1]:
from genepy.utils import helper as h

# to comment in your case
from taigapy import TaigaClient
tc = TaigaClient()

from celligner import Celligner
import pandas as pd
# to comment in your case
from depmapomics import tracker as track
#autoreload
%load_ext autoreload
%autoreload 2
#output
from bokeh.plotting import output_notebook
output_notebook()

from celligner.params import TISSUE_COLOR

/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "l2_norm" failed type inference due to: No implementation of function Function(<function norm at 0x7f9ec40322f0>) found for signature:
 
 >>> norm(x=array(float32, 2d, A), axis=Literal[int](1))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'norm_impl': File: numba/np/linalg.py: Line 2352.
    With argument(s): '(x=array(float32, 2d, A), axis=int64)':
   Rejected as the implementation raised a specific error:
     TypeError: norm_impl() got an unexpected keyword argument 'x'
  raised from /home/jeremie/miniconda3/lib/python3.7/site-packages/numba/core/typing/templates.py:722

During: resolving callee type: Function(<function norm at 0x7f9ec40322f0>)
During: typing of call at /home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py (16)


File "../celligner/mnnpy/mnnpy/utils.py",

Cython module _utils not initialized. Fallback to python.


Loading BokehJS ...

In [52]:
my_alligner = Celligner()
my_alligner.load('../temp/22Q1')

fetching gene names from biomart cache
using only usefull genes


## adding CCLF datasset

In [3]:
import dalmatian as dm

In [28]:
file = dm.WorkspaceManager('nci-mimoun-bi-org/CCLF_RNA_2_0').get_sample_sets().loc['CCLF7', 'genes_tpm']
cclf = pd.read_csv(file, sep="\t").drop(columns='transcript_id(s)').set_index('gene_id', drop=True).T

In [29]:
for i in h.dups([i.split('.')[0] for i in cclf.columns]):
    cols = cclf.columns[cclf.columns.str.contains(i)]
    cclf[i] = cclf[cols].mean(1)
    cclf = cclf.drop(columns=cols)
cclf.columns = [i.split('.')[0] for i in cclf.columns]

In [53]:
# if you want to align to both CCLE and TCGA, you can ask celligner to consider the two (fit + _pre-transformed_ transform datasets) as a fit dataset by calling:
# my_alligner.putAllToFit()

# you can add your dataset as a dataset to be aligned to, by puting it in fit:
# my_alligner.addToFit(yourdataset).transform()
# /!\ need to already have a transform dataset (if you loaded the example model, this is TCGA)

# you can add your dataset as one to align, by putting it in transform:
# my_alligner.addToTransform(yourdataset)
# /!\ need to already have a fit dataset (if you loaded the example model, this is CCLE)

# if your dataset is small enough it might actually not work well to put it in transform it seems!
# if your dataset is small and similar enough, you can add the parameter dotransform=False (or dofit=False) so that it doesn't fully retransforms or refit but uses cached computation instead.
my_alligner.priotize_fit=False
my_alligner.make_plots=False
my_alligner.putAllToFit(redo_diff=False)
my_alligner.mnn_kwargs.update({'k1': 20})
_ = my_alligner.transform(cclf, recompute_contamination=False, _rerun=False)

clustering...
         Falling back to preprocessing with `sc.pp.pca` and default params.
done
looking at 7 samples.
found 18049 common genes
creating a transform input..
doing differential expression analysis on the clusters..
regressing out the cPCA components..
doing the MNN analysis using Marioni et al. method..
  Looking for MNNs...
  Found 140 mutual nearest neighbors.
done


## making it portal-like

In [39]:
set(my_alligner.annotations.cell_type)

{'CCLE cell line', 'PDX', 'TCGA tumor', 'met500 tumor', 'ped PDX'}

In [55]:
my_alligner.fit_annotations

,tissue_type,disease_type,cell_type
ACH-001113,lung,NSCLC,CCLE cell line
ACH-001289,soft_tissue,ATRT,CCLE cell line
ACH-001339,skin,melanoma,CCLE cell line
ACH-001538,bile_duct,cholangiocarcinoma,CCLE cell line
ACH-000242,urinary_tract,bladder_carcinoma,CCLE cell line
...,...,...,...
PALKTY,blood,ALL,ped PDX
PALLSD,blood,ALL,ped PDX
PALNTB,blood,ALL,ped PDX
PALTWS,blood,ALL,ped PDX


In [ ]:
my_alligner.fit_annotationsnotationsnotationsnotationsnnotations

In [56]:
my_alligner.transform_annotations

,cell_type,disease_type,tissue_type
RP-571_CCLF_KL9001T_v2_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_KL9002T_v2_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_KL9003T_v2_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_NB1006-1_v1_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_NB1006T-2_v1_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_NB1006T-3_v1_RNA_OnPrem,5.0,5.0,5.0
RP-571_CCLF_NB1006T-4_v1_RNA_OnPrem,5.0,5.0,5.0


In [68]:
imp = np.zeros(len(my_alligner.fit_annotations)+len(my_alligner.transform_annotations))
imp[:len(my_alligner.fit_annotations)] = 2
colors = np.zeros(len(my_alligner.fit_annotations)+len(my_alligner.transform_annotations))
colors[:len(my_alligner.fit_annotations)] = 1
my_alligner.plot(importance = imp, colors=colors, folder='./cclf_analysis')

making plot...


/home/jeremie/miniconda3/lib/python3.7/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/jeremie/miniconda3/lib/python3.7/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


RuntimeError: To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')

In [69]:
my_alligner.plot(color_column="tissue_type", colortable={**TISSUE_COLOR, **{5: '#000000'}}, rerun=False, shape=my_alligner.annotations.cell_type.values, folder='./cclf_color_analysis')

making plot...


RuntimeError: To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')

In [70]:
nn = my_alligner.getKNN(ofcell=5, incell="CCLE cell line")

In [71]:
nn

,0,1,2,3,4,dist0,dist1,dist2,dist3,dist4,pred,prob
RP-571_CCLF_KL9001T_v2_RNA_OnPrem,ACH-000780,ACH-000431,ACH-000904,ACH-000729,ACH-000523,22031.1,22031.7,22032.2,22032.2,22033,lung,1
RP-571_CCLF_KL9002T_v2_RNA_OnPrem,ACH-000431,ACH-000523,ACH-001338,ACH-000904,ACH-000780,19496.6,19496.9,19497.1,19497.3,19497.3,lung,0.8
RP-571_CCLF_KL9003T_v2_RNA_OnPrem,ACH-001338,ACH-001578,ACH-001367,ACH-000431,ACH-000804,21211.1,21211.7,21211.7,21211.7,21211.9,peripheral_nervous_system,0.6
RP-571_CCLF_NB1006-1_v1_RNA_OnPrem,ACH-001376,ACH-001855,ACH-001696,ACH-001494,ACH-001378,10821,10821.2,10822,10822.1,10822.1,pancreas,0.4
RP-571_CCLF_NB1006T-2_v1_RNA_OnPrem,ACH-001388,ACH-001376,ACH-001400,ACH-001696,ACH-001378,37599.4,37599.8,37600.2,37601.9,37602.3,breast,0.2
RP-571_CCLF_NB1006T-3_v1_RNA_OnPrem,ACH-002531,ACH-001839,ACH-001835,ACH-001847,ACH-000414,16829.8,16830.8,16831.6,16831.7,16831.7,lung,0.4
RP-571_CCLF_NB1006T-4_v1_RNA_OnPrem,ACH-001388,ACH-000415,ACH-001400,ACH-001696,ACH-001525,24686.2,24688.8,24689.7,24689.8,24690,breast,0.2


In [73]:
nn = my_alligner.getKNN(ofcell=5, incell="TCGA tumor")

In [82]:
nn[['pred','prob']]

,pred,prob
RP-571_CCLF_KL9001T_v2_RNA_OnPrem,central_nervous_system,1
RP-571_CCLF_KL9002T_v2_RNA_OnPrem,central_nervous_system,1
RP-571_CCLF_KL9003T_v2_RNA_OnPrem,central_nervous_system,1
RP-571_CCLF_NB1006-1_v1_RNA_OnPrem,gastric,0.4
RP-571_CCLF_NB1006T-2_v1_RNA_OnPrem,central_nervous_system,0.2
RP-571_CCLF_NB1006T-3_v1_RNA_OnPrem,lung,1
RP-571_CCLF_NB1006T-4_v1_RNA_OnPrem,upper_aerodigestive,0.2


## saving and exporting

In [77]:
def exportCellignerToPortal(my_alligner):
    return pd.concat([my_alligner.annotations, pd.DataFrame(data=my_alligner.umap_reduced, index=my_alligner.annotations.index, columns=['umap1', 'umap2']),], axis=1)

In [79]:
exportCellignerToPortal(my_alligner).to_csv('../temp/cclf_rerun_celligner_aligned_all.csv')

In [81]:
new_dataset_id = tc.update_dataset(
    "celligner-multi-dataset-alignment-5403",
    changes_description="adding the cclf run with 7 samples",
    upload_files=[
        {
            "path": "../temp/cclf_rerun_celligner_aligned_all.csv",
            "format": "TableCSV", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    add_all_existing_files=True, # If True, will add all files from the base dataset version, except files with the same names as those in upload_files or add_taiga_ids
)

No dataset version provided. Using version 2.
Uploading cclf_rerun_celligner_aligned_all to S3
Finished uploading cclf_rerun_celligner_aligned_all to S3
Uploading cclf_rerun_celligner_aligned_all to Taiga
Finished uploading cclf_rerun_celligner_aligned_all to Taiga
Linking virtual file celligner-multi-dataset-alignment-5403.2/celligner_aligned_all
Dataset version created. Access it directly with this url: https://cds.team/taiga/dataset_version/63bd1994ce434e0899ff6b2636f00805


## computing metrics 

In [43]:
import scib
from anndata import AnnData

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
adata = AnnData(my_alligner.fit_input.append(my_alligner.transform_input), my_alligner.fit_annotations.append(my_alligner.transform_annotations))

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
adata.obs['cell_type'] = adata.obs['cell_type'].astype("category")
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype("category")

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
